In [1]:
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType # type: ignore
from transformers import BitsAndBytesConfig
import torch
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold,StratifiedGroupKFold
import numpy as np
import random
import pandas as pd
import os
from __future__ import annotations
from datasets import Dataset
from transformers import AutoTokenizer, LlamaForSequenceClassification, MistralForSequenceClassification
from transformers import DataCollatorWithPadding
from sklearn.metrics import log_loss
from transformers import TrainingArguments, Trainer
from shutil import rmtree
from scipy.special import softmax
import gc
from pathlib import Path

In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['TRANSFORMERS_CACHE'] = '///mnt/c/Personal/Competitions/HFCache/'
os.environ['HF_HOME'] = '///mnt/c/Personal/Competitions/HFCache/'

In [3]:
class Config:
    # General settings
    EXP_NAME = 'nb005'
    competition_name = 'h2O_llm'
    seed = 2022 #42
    debug = False
    train = True
    n_fold = 5
    TARGET_MODEL = "mistralai/Mistral-7B-Instruct-v0.1" #"mistralai/Mistral-7B-v0.1"
    DEBUG = False
    max_len = 2048
    
CFG = Config()

In [4]:
def seed_everything(seed, use_cuda = True):
    np.random.seed(seed) # cpu vars
    torch.manual_seed(seed) # cpu  vars
    random.seed(seed) # Python
    os.environ['PYTHONHASHSEED'] = str(seed) # Python hash building
    if use_cuda:
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

seed_everything(CFG.seed)

In [5]:
data_dir = "///mnt/c/Personal/Competitions/Kaggle/h2oai-predict-the-llm/"
OUTPUT_DIR = f'///mnt/c/Personal/Competitions/Kaggle/h2oai-predict-the-llm/runs/' + CFG.EXP_NAME + "/"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

train = pd.read_csv(data_dir + "train.csv").rename(columns={'target': 'label'})
test = pd.read_csv(data_dir + "test.csv")
sample_submission = pd.read_csv(data_dir + "sample_submission.csv")

In [6]:
train.head()

,Question,Response,label
0,Explain the concept of coevolution.,Coevolution is a biological process that occur...,3
1,Is it possible that recurring fever and chills...,"Yes, recurring fever and chills can be a sympt...",4
2,Evaluate the expression 3!,The expression 3! represents the factorial of ...,1
3,What are the roles of different types of RNA i...,1. Messenger RNA (mRNA): mRNA carries genetic ...,3
4,What is the role of gene flow in population ge...,Gene flow refers to the movement of individual...,3


In [7]:
# train['Question'] = train['Question'].str.replace('\n', '')
# train['Response'] = train['Response'].str.replace('\n', '')
# test['Question'] = test['Question'].str.replace('\n', '')
# test['Response'] = test['Response'].str.replace('\n', '')


In [8]:
gkf = StratifiedGroupKFold(n_splits=CFG.n_fold,shuffle=True,random_state=CFG.seed)

for i, (_, val_index) in enumerate(gkf.split(train, train["label"],groups=train['Question'])):
    train.loc[val_index, "fold"] = i

train = train.fillna("NA")
test = test.fillna("NA")

In [9]:
train['all_text'] = 'Question: ' + train.Question + '; Answer: ' + train.Response
test['all_text'] = 'Question: ' + test.Question + '; Answer: ' + test.Response

In [10]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    target_modules=[
        "q_proj",
        "v_proj"
    ],
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [11]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = softmax(predictions,axis=1)
    print(predictions)
    logloss_val = log_loss(labels, predictions)
    
    return {
        "logloss": logloss_val,
    }

In [12]:
def run(fld):
    
    tokenizer = AutoTokenizer.from_pretrained(CFG.TARGET_MODEL, use_fast=False)
    tokenizer.pad_token = tokenizer.eos_token
    # LlamaForSequenceClassification(
    base_model = MistralForSequenceClassification.from_pretrained(
        CFG.TARGET_MODEL,
        num_labels=7,
        cache_dir='///mnt/c/Personal/Competitions/HFCache/',
        quantization_config=bnb_config,
        device_map={"":0})
    
    # base_model.config.pretraining_tp = 1 # 1 is 7b
    base_model.config.pad_token_id = tokenizer.pad_token_id
    model = get_peft_model(base_model, peft_config)
    model.print_trainable_parameters()

    train_df = train[train['fold']!=fld].reset_index(drop=True)
    valid_df = train[train['fold']==fld].reset_index(drop=True)

    print('train shape:',train_df.shape, 'valid shape:',valid_df.shape)
    # from pandas
    train_ds = Dataset.from_pandas(train_df)
    valid_ds = Dataset.from_pandas(valid_df)

    def preprocess_function(examples, max_length=CFG.max_len):
        return tokenizer(examples["all_text"], 
                         truncation=True, 
                         max_length=max_length, 
                         padding=True)

    train_tokenized_ds = train_ds.map(preprocess_function, batched=True)
    valid_tokenized_ds = valid_ds.map(preprocess_function, batched=True)

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")    
    model_fold_dir = os.path.join(OUTPUT_DIR,str(fld)) 
    
    training_args = TrainingArguments(
        output_dir=model_fold_dir,
        learning_rate=3e-4,#5e-5,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=32,
        max_grad_norm=0.3,
        optim='paged_adamw_32bit',
        lr_scheduler_type="cosine",
        num_train_epochs=5,
        weight_decay=0.01,
        evaluation_strategy="steps",
        save_strategy="steps",
        load_best_model_at_end=True,
        push_to_hub=False,
        warmup_steps=100,
        eval_steps=50,
        save_steps = 50,
        logging_steps=50,
        report_to='none' # if DEBUG else 'wandb',
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized_ds,
        eval_dataset=valid_tokenized_ds,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    
    trainer.train()

    trainer.save_model(output_dir=str(model_fold_dir))
    
    for path in Path(training_args.output_dir).glob("checkpoint-*"):
        if path.is_dir():
            rmtree(path)
            
    del trainer, model, base_model

    for i in range(5):
        torch.cuda.empty_cache()
        gc.collect()

In [13]:
for f in range(5):
    run(f)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-Instruct-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 6,873,088 || all params: 7,117,533,184 || trainable%: 0.09656559122829936
train shape: (3178, 5) valid shape: (798, 5)


Map:   0%|          | 0/3178 [00:00<?, ? examples/s]

Map:   0%|          | 0/798 [00:00<?, ? examples/s]

...

trainable params: 6,873,088 || all params: 7,117,533,184 || trainable%: 0.09656559122829936
train shape: (3185, 5) valid shape: (791, 5)


Map:   0%|          | 0/3185 [00:00<?, ? examples/s]

Map:   0%|          | 0/791 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Logloss
50,5.055400,2.103516,1.970110
100,1.515400,1.218750,1.214971
150,1.015400,1.085938,1.083193
200,0.921800,1.141602,1.125524
250,0.574300,1.016602,0.995805
300,0.555500,0.923340,0.924513
350,0.217900,1.425781,1.326373
400,0.158900,1.500000,1.321854
450,0.025400,2.128906,1.550182


[[0.05463  0.00801  0.2825   ... 0.6025   0.03513  0.0127  ]
 [0.01968  0.0009   0.02263  ... 0.477    0.03363  0.4363  ]
 [0.08716  0.00493  0.1628   ... 0.2496   0.1683   0.3193  ]
 ...
 [0.1538   0.01714  0.1248   ... 0.2786   0.04556  0.3704  ]
 [0.01119  0.563    0.0837   ... 0.0488   0.001907 0.2876  ]
 [0.1675   0.3242   0.04156  ... 0.02277  0.053    0.1808  ]]
[[1.0736e-01 4.1089e-01 1.9226e-01 ... 2.3132e-01 1.5774e-03 1.8524e-02]
 [2.0630e-01 1.0565e-01 2.1082e-01 ... 1.7664e-01 2.8091e-02 1.9507e-01]
 [2.9053e-01 7.7698e-02 1.5186e-01 ... 1.2732e-01 1.5612e-03 2.1692e-01]
 ...
 [3.0542e-01 6.0852e-02 1.4673e-01 ... 1.9409e-01 1.1454e-03 2.1484e-01]
 [1.9653e-02 5.0879e-01 3.0777e-02 ... 5.5847e-03 1.3185e-04 4.2993e-01]
 [2.4399e-02 2.3779e-01 1.8387e-02 ... 1.3494e-03 7.8011e-03 3.0908e-01]]
[[4.9011e-02 9.8953e-03 2.9224e-01 ... 6.3916e-01 2.5826e-03 2.4486e-04]
 [1.6125e-01 2.1011e-02 2.3047e-01 ... 3.9966e-01 1.3574e-01 2.9694e-02]
 [1.8408e-01 5.7129e-02 2.8735e-01 ...

In [15]:
gc.collect()

0

In [17]:
torch.cuda.empty_cache()

In [23]:
def preprocess_function(examples, max_length=CFG.max_len):
    return tokenizer(examples["all_text"], 
                     truncation=True, 
                     max_length=max_length, 
                     padding=True)
tokenizer = AutoTokenizer.from_pretrained(CFG.TARGET_MODEL, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

test_ds = Dataset.from_pandas(test)
test_tokenized_ds = test_ds.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

In [48]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [49]:
base_model = MistralForSequenceClassification.from_pretrained(
    CFG.TARGET_MODEL,
    num_labels=7,
    cache_dir='///mnt/c/Personal/Competitions/HFCache/',
    quantization_config=bnb_config,
    device_map={"":0})

base_model.config.pad_token_id = tokenizer.pad_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-Instruct-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
import scipy
out = np.zeros((1001, 7))
for f in (range(5)):
    print(f'----------- Fold: {f} ----------')
    model = PeftModel.from_pretrained(base_model, f'///mnt/c/Personal/Competitions/Kaggle/h2oai-predict-the-llm/runs/nb005/{f}/')

    trainer = Trainer(model=model,
                      tokenizer=tokenizer,
                      data_collator=data_collator)
    pred_output = trainer.predict(test_tokenized_ds)
    logits = pred_output.predictions
    probits = scipy.special.softmax(logits,1)

    out += probits/5
    del trainer, model
    torch.cuda.empty_cache()    
    gc.collect()

----------- Fold: 0 ----------


----------- Fold: 1 ----------


----------- Fold: 2 ----------


----------- Fold: 3 ----------


----------- Fold: 4 ----------


In [52]:
out

array([[3.57788086e-01, 2.13571787e-02, 1.42257690e-01, ...,
        3.51562500e-01, 5.91926575e-02, 5.65297604e-02],
       [1.32858753e-04, 3.08442116e-03, 3.81469727e-05, ...,
        1.68383121e-04, 9.94873047e-01, 1.45876408e-03],
       [3.57627869e-07, 1.60932541e-06, 3.57627869e-07, ...,
        4.17232513e-07, 9.99755859e-01, 1.37090683e-06],
       ...,
       [1.96762085e-01, 1.01788521e-01, 2.78015137e-01, ...,
        3.66149902e-01, 1.05845928e-02, 3.69189978e-02],
       [2.90420532e-01, 3.60937119e-02, 2.08209991e-01, ...,
        4.19372559e-01, 1.63280964e-03, 3.95431519e-02],
       [1.06906891e-03, 2.99870968e-03, 5.77747822e-04, ...,
        1.06978416e-03, 1.09314919e-04, 9.93896484e-01]])

In [55]:
sample_submission.iloc[:,1:] = out

In [58]:
OUTPUT_DIR

'///mnt/c/Personal/Competitions/Kaggle/h2oai-predict-the-llm/runs/nb005/'

In [59]:
sample_submission.to_csv(f'{OUTPUT_DIR}/submit.csv',index=False)